# Gather relevant data 

## Webscrape demographic data 

Import Relevant packages

In [1]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

Checks to confirm page exists

In [4]:
url = 'https://www.zip-codes.com{}'
columbus_branch ='/city/oh-columbus.asp'
# reinforces integrity of the data collection
try:
    response = requests.get(url.format(columbus_branch))
    print(response)
except:
    print('Page does not exist')

<Response [200]>


Gathers relevant data on the page

In [5]:
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    tbl = soup.table.tr.find_all('td')[1].find_all('table')[1]
    print(tbl)

<table border="0" cellpadding="0" cellspacing="0" class="statTable" id="tblZIP" width="99%">
<tr><td class="label"><strong>ZIP Code</strong></td><td><strong>Type</strong></td><td><strong>County</strong></td><td><strong>Population</strong></td><td><strong>Area Code(s)</strong></td></tr>
<tr><td><a href="/zip-code/43081/zip-code-43081.asp" style="text-decoration:underline;" title="ZIP Code 43081, WESTERVILLE, OH">ZIP Code 43081</a></td><td>Standard</td><td><a href="/county/oh-franklin.asp">Franklin</a></td><td>55,991</td><td><a href="/area-code/area-code-614.asp" title="Area Code 614">614</a></td></tr><tr><td><a href="/zip-code/43085/zip-code-43085.asp" style="text-decoration:underline;" title="ZIP Code 43085, COLUMBUS, OH">ZIP Code 43085</a></td><td>Standard</td><td><a href="/county/oh-franklin.asp">Franklin</a></td><td>23,258</td><td><a href="/area-code/area-code-614.asp" title="Area Code 614">614</a> / <a href="/area-code/area-code-380.asp" title="Area Code 380">380</a> / <a href="/ar

Parses through and gathers data

In [9]:
zip_url = '/zip-code/{}/zip-code-{}.asp'
for row in tbl.find_all('tr'):
    #checks for header
    if row.find_all('td')[0].string != 'ZIP Code':
        if row.find_all('td')[1].string == 'Standard':
            zip_code = row.find_all('td')[0].string
            try:
                zip_response = requests.get(url.format(zip_url.format(zip_code,zip_code)))
                print(zip_response)
            except:
                print('URL not found')
            finally:
                print(zip_code)
            

<Response [200]>
URL not found
URL not found
<Response [200]>
<Response [200]>
URL not found
URL not found
URL not found
URL not found
<Response [200]>
URL not found
URL not found
<Response [200]>
<Response [200]>
<Response [200]>
URL not found
<Response [200]>
URL not found
<Response [200]>
URL not found
URL not found
URL not found
URL not found
<Response [200]>
URL not found
<Response [200]>
URL not found
<Response [200]>
URL not found
URL not found
<Response [200]>


## Location data for Hospitals and Pharmacies

Get relevant packages

In [ ]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

Credentials for foursquare_API

In [ ]:
# The code was removed by Watson Studio for sharing.